# Redes Generativas

Redes Neurais Recorrentes (RNNs) e suas variantes com células controladas, como Células de Memória de Longo Prazo (LSTMs) e Unidades Recorrentes Controladas (GRUs), fornecem um mecanismo para modelagem de linguagem, ou seja, elas podem aprender a ordem das palavras e fornecer previsões para a próxima palavra em uma sequência. Isso nos permite usar RNNs para **tarefas generativas**, como geração de texto comum, tradução automática e até mesmo legendagem de imagens.

Na arquitetura de RNN que discutimos na unidade anterior, cada unidade RNN produzia o próximo estado oculto como saída. No entanto, também podemos adicionar outra saída a cada unidade recorrente, o que nos permitiria gerar uma **sequência** (que tem o mesmo comprimento da sequência original). Além disso, podemos usar unidades RNN que não aceitam uma entrada em cada etapa, apenas recebem um vetor de estado inicial e, em seguida, produzem uma sequência de saídas.

Neste notebook, vamos nos concentrar em modelos generativos simples que nos ajudam a gerar texto. Para simplificar, vamos construir uma **rede em nível de caracteres**, que gera texto letra por letra. Durante o treinamento, precisamos pegar algum corpus de texto e dividi-lo em sequências de letras.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Construindo um vocabulário de caracteres

Para construir uma rede generativa em nível de caracteres, precisamos dividir o texto em caracteres individuais em vez de palavras. A camada `TextVectorization` que usamos anteriormente não consegue fazer isso, então temos duas opções:

* Carregar o texto manualmente e fazer a tokenização "manualmente", como neste [exemplo oficial do Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Usar a classe `Tokenizer` para tokenização em nível de caracteres.

Vamos seguir com a segunda opção. `Tokenizer` também pode ser usado para tokenizar em palavras, então deve ser possível alternar facilmente entre tokenização em nível de caracteres e em nível de palavras.

Para realizar a tokenização em nível de caracteres, precisamos passar o parâmetro `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Também queremos usar um token especial para denotar **fim da sequência**, que chamaremos de `<eos>`. Vamos adicioná-lo manualmente ao vocabulário:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Treinando uma RNN generativa para criar títulos

A maneira como treinaremos a RNN para gerar títulos de notícias é a seguinte. A cada etapa, pegaremos um título, que será alimentado em uma RNN, e para cada caractere de entrada pediremos à rede que gere o próximo caractere de saída:

![Imagem mostrando um exemplo de geração de RNN da palavra 'HELLO'.](../../../../../translated_images/pt-BR/rnn-generate.56c54afb52f9781d.webp)

Para o último caractere da nossa sequência, pediremos à rede que gere o token `<eos>`.

A principal diferença da RNN generativa que estamos usando aqui é que pegaremos a saída de cada etapa da RNN, e não apenas da célula final. Isso pode ser alcançado especificando o parâmetro `return_sequences` na célula da RNN.

Assim, durante o treinamento, a entrada para a rede será uma sequência de caracteres codificados de algum comprimento, e a saída será uma sequência do mesmo comprimento, mas deslocada por um elemento e terminada com `<eos>`. O minibatch consistirá de várias dessas sequências, e precisaremos usar **padding** para alinhar todas as sequências.

Vamos criar funções que transformarão o conjunto de dados para nós. Como queremos preencher as sequências no nível do minibatch, primeiro agruparemos o conjunto de dados chamando `.batch()`, e depois usaremos `map` para realizar a transformação. Portanto, a função de transformação receberá um minibatch inteiro como parâmetro:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Algumas coisas importantes que fazemos aqui:
* Primeiro, extraímos o texto real do tensor de string
* `text_to_sequences` converte a lista de strings em uma lista de tensores inteiros
* `pad_sequences` ajusta esses tensores ao seu comprimento máximo
* Por fim, codificamos todos os caracteres em one-hot, além de realizar o deslocamento e adicionar `<eos>`. Em breve veremos por que precisamos de caracteres codificados em one-hot

No entanto, essa função é **Pythonic**, ou seja, ela não pode ser automaticamente traduzida para o grafo computacional do Tensorflow. Receberemos erros se tentarmos usar essa função diretamente na função `Dataset.map`. Precisamos encapsular essa chamada Pythonic usando o wrapper `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Nota**: Diferenciar entre funções de transformação Pythonic e funções de transformação do Tensorflow pode parecer um pouco complexo demais, e você pode estar se perguntando por que não transformamos o conjunto de dados usando funções padrão do Python antes de passá-lo para `fit`. Embora isso definitivamente possa ser feito, usar `Dataset.map` tem uma grande vantagem, pois o pipeline de transformação de dados é executado usando o grafo computacional do Tensorflow, que aproveita os cálculos na GPU e minimiza a necessidade de transferir dados entre CPU/GPU.

Agora podemos construir nossa rede geradora e começar o treinamento. Ela pode ser baseada em qualquer célula recorrente que discutimos na unidade anterior (simples, LSTM ou GRU). Em nosso exemplo, usaremos LSTM.

Como a rede recebe caracteres como entrada e o tamanho do vocabulário é relativamente pequeno, não precisamos de uma camada de embedding; a entrada codificada em one-hot pode ir diretamente para a célula LSTM. A camada de saída será um classificador `Dense` que converterá a saída do LSTM em números de tokens codificados em one-hot.

Além disso, como estamos lidando com sequências de comprimento variável, podemos usar a camada `Masking` para criar uma máscara que ignorará a parte preenchida da string. Isso não é estritamente necessário, porque não estamos muito interessados em tudo que vai além do token `<eos>`, mas usaremos essa camada para ganhar alguma experiência com esse tipo de camada. O `input_shape` será `(None, vocab_size)`, onde `None` indica a sequência de comprimento variável, e o formato de saída também será `(None, vocab_size)`, como você pode ver no `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Gerando saída

Agora que treinamos o modelo, queremos usá-lo para gerar alguma saída. Antes de tudo, precisamos de uma maneira de decodificar o texto representado por uma sequência de números de tokens. Para isso, poderíamos usar a função `tokenizer.sequences_to_texts`; no entanto, ela não funciona bem com tokenização em nível de caracteres. Portanto, vamos pegar um dicionário de tokens do tokenizer (chamado `word_index`), construir um mapa reverso e escrever nossa própria função de decodificação:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Agora, vamos realizar a geração. Começaremos com uma string `start`, codificaremos ela em uma sequência `inp`, e então, a cada etapa, chamaremos nossa rede para inferir o próximo caractere.

A saída da rede `out` é um vetor de `vocab_size` elementos que representa as probabilidades de cada token, e podemos encontrar o número do token mais provável usando `argmax`. Em seguida, adicionamos esse caractere à lista de tokens gerados e continuamos com a geração. Esse processo de gerar um caractere é repetido `size` vezes para gerar o número necessário de caracteres, e encerramos antecipadamente quando o `eos_token` é encontrado.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Amostrando saída durante o treinamento

Como não temos métricas úteis como *precisão*, a única maneira de verificar se nosso modelo está melhorando é **amostrando** strings geradas durante o treinamento. Para isso, usaremos **callbacks**, ou seja, funções que podemos passar para a função `fit`, e que serão chamadas periodicamente durante o treinamento.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Este exemplo já gera um texto bastante bom, mas pode ser melhorado de várias maneiras:

* **Mais texto**. Usamos apenas títulos para nossa tarefa, mas você pode querer experimentar com texto completo. Lembre-se de que RNNs não lidam muito bem com sequências longas, então faz sentido dividi-las em frases mais curtas ou sempre treinar com um comprimento de sequência fixo de algum valor predefinido `num_chars` (por exemplo, 256). Você pode tentar modificar o exemplo acima para essa arquitetura, usando o [tutorial oficial do Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) como inspiração.

* **LSTM com várias camadas**. Faz sentido tentar 2 ou 3 camadas de células LSTM. Como mencionamos na unidade anterior, cada camada de LSTM extrai certos padrões do texto, e no caso de um gerador em nível de caracteres, podemos esperar que o nível mais baixo do LSTM seja responsável por extrair sílabas, e os níveis mais altos - por palavras e combinações de palavras. Isso pode ser implementado simplesmente passando o parâmetro de número de camadas para o construtor do LSTM.

* Você também pode querer experimentar com **unidades GRU** e ver quais apresentam melhor desempenho, além de **diferentes tamanhos de camadas ocultas**. Camadas ocultas muito grandes podem resultar em overfitting (por exemplo, a rede aprenderá o texto exato), e tamanhos menores podem não produzir bons resultados.


## Geração de texto suave e temperatura

Na definição anterior de `generate`, sempre escolhemos o caractere com a maior probabilidade como o próximo caractere no texto gerado. Isso resultava no fato de que o texto frequentemente "ciclava" entre as mesmas sequências de caracteres repetidamente, como neste exemplo:
```
today of the second the company and a second the company ...
```

No entanto, se analisarmos a distribuição de probabilidade para o próximo caractere, pode ser que a diferença entre algumas das maiores probabilidades não seja tão grande. Por exemplo, um caractere pode ter probabilidade de 0,2, enquanto outro tem 0,19, etc. Por exemplo, ao procurar o próximo caractere na sequência '*play*', o próximo caractere pode ser tanto um espaço quanto **e** (como na palavra *player*).

Isso nos leva à conclusão de que não é sempre "justo" selecionar o caractere com maior probabilidade, pois escolher o segundo mais provável ainda pode nos levar a um texto significativo. É mais sensato **amostrar** caracteres a partir da distribuição de probabilidade fornecida pela saída da rede.

Essa amostragem pode ser feita usando a função `np.multinomial`, que implementa a chamada **distribuição multinomial**. Uma função que implementa essa geração de texto **suave** está definida abaixo:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Introduzimos mais um parâmetro chamado **temperatura**, que é usado para indicar o quão rigidamente devemos nos ater à maior probabilidade. Se a temperatura for 1.0, fazemos uma amostragem multinomial justa, e quando a temperatura vai para o infinito - todas as probabilidades se tornam iguais, e selecionamos o próximo caractere aleatoriamente. No exemplo abaixo, podemos observar que o texto se torna sem sentido quando aumentamos demais a temperatura, e se assemelha a um texto "ciclado" gerado rigidamente quando se aproxima de 0.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
